# PCAD notebook 6

This notebook loads the pickle files produced by `pcad_5_combine_coverage_calculate`, adds print locations to e lines in each dataframe, then writes out all dataframes to a multi-worksheet Excel file. Congratulations, you made it.

Required files/inputs:
- `multi_loc_100*.pkl` files produced by PCAD notebook 5 (8 files)
- `single_loc_100*.pkl` files produced by PCAD notebook 5 (8 files)

Outputs:
- `pcad_final_{date}.xlsx` with 1 worksheet per dataframe

In [3]:
import pandas as pd
from datetime import date
today = str(date.today()).replace('-','')

In [4]:
df_mlsv = pd.read_pickle('multi_loc_100single-vendor-data.pkl')
df_mlmv = pd.read_pickle('multi_loc_100multi-vendor-data.pkl')
df_ml_els = pd.read_pickle('multi_loc_100_vendor_Elsevier.pkl')
df_ml_sage = pd.read_pickle('multi_loc_100_vendor_SAGE.pkl')
df_ml_spr = pd.read_pickle('multi_loc_100_vendor_Springer.pkl')
df_ml_tf = pd.read_pickle('multi_loc_100_vendor_Taylor & Francis.pkl')
df_ml_w = pd.read_pickle('multi_loc_100_vendor_Wiley.pkl')
df_ml_o = pd.read_pickle('multi_loc_100_vendor_other_and_JSTOR.pkl')

In [5]:
multi_dfs = [df_mlsv, df_mlmv, df_ml_els, df_ml_sage, df_ml_spr, df_ml_tf, df_ml_w, df_ml_o]
for frame in multi_dfs:
    print(frame.shape)

(120, 43)
(6, 43)
(80, 43)
(2, 43)
(4, 43)
(2, 43)
(6, 43)
(26, 43)


In [6]:
#Placeholder cell -- use to drop any empty dataframes from multi_dfs

In [9]:
df_slsv = pd.read_pickle('single_loc_100single-vendor-data.pkl')
df_slmv = pd.read_pickle('single_loc_100multi-vendor-data.pkl')
df_sl_els = pd.read_pickle('single_loc_100_vendor_Elsevier.pkl')
df_sl_sage = pd.read_pickle('single_loc_100_vendor_SAGE.pkl')
df_sl_spr = pd.read_pickle('single_loc_100_vendor_Springer.pkl')
df_sl_tf = pd.read_pickle('single_loc_100_vendor_Taylor & Francis.pkl')
df_sl_w = pd.read_pickle('single_loc_100_vendor_Wiley.pkl')
df_sl_o = pd.read_pickle('single_loc_100_vendor_other_and_JSTOR.pkl')

In [10]:
single_dfs = [df_slsv, df_slmv, df_sl_els, df_sl_sage, df_sl_spr, df_sl_tf, df_sl_w, df_sl_o]
for frame in single_dfs:
    print(frame.shape)

(536, 43)
(30, 43)
(256, 43)
(14, 43)
(66, 43)
(14, 43)
(60, 43)
(126, 43)


In [11]:
#Placeholder cell -- use to drop any empty dataframes from single_dfs

In [14]:
def add_p_locs_to_e_lines(df):
    df['new_locs'] = df['locs']
    df['new_locs'].fillna('NOLOCS', inplace=True)
    df['new_locs'].astype(str)
    df['final_group_id'].astype(int)
    locs_df = df.filter(['final_group_id', 'new_locs'], axis=1)
    locs_df = locs_df[(locs_df['new_locs']) != 'NOLOCS']
    df_locs_all_rows = pd.merge(df, locs_df, how='left', on='final_group_id')
    df_locs_clean = df_locs_all_rows.drop('new_locs_x', axis=1)
    df_locs_clean.rename(columns={'new_locs_y' : 'P_locs'}, inplace=True)
    return df_locs_clean

In [26]:
all_dfs = []
for frame in multi_dfs:
    clean_df = add_p_locs_to_e_lines(frame)
    all_dfs.append(clean_df)

In [27]:
len(all_dfs)

8

In [28]:
for frame in single_dfs:
    clean_df = add_p_locs_to_e_lines(frame)
    all_dfs.append(clean_df)

In [29]:
len(all_dfs)

16

In [30]:
with pd.ExcelWriter(f'pcad_final_{today}.xlsx') as writer:
    i = 0
    for df in all_dfs:
        sheet = f'Sheet{str(i)}'
        df.to_excel(writer, sheet_name=sheet, index=False)
        i +=1    

*Note: you will still have to rename the worksheets in the Excel file manually. TODO, someday: code this bit.* 